In [41]:
import json
import requests

# API KEY INPUT HERE
apikey = ""

def task1(stocks, apikey):
    url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": ",".join(stocks)}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    if 'quoteResponse' in data and 'result' in data['quoteResponse']:
        for stock in data['quoteResponse']['result']:
            symbol = stock.get('symbol', 'N/A')
            company = stock.get('longName', 'N/A')
            price = stock.get('regularMarketPrice', 'N/A')

            if isinstance(price, (int, float)):
                price = f"${price:.2f}"
            else:
                price = "N/A"

            print(f"Stock Ticker: {symbol}, Company: {company}, Current Market Price: {price}")
    else:
        print("Invalid stock symbol")

def task2_1(symbol, module, apikey):
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    querystring = {"modules": "summaryDetail,financialData"}
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()

    if 'quoteSummary' in data and 'result' in data['quoteSummary']:
        stock_info = data['quoteSummary']['result'][0]
        summary_detail = stock_info.get("summaryDetail", {})
        financial_data = stock_info.get("financialData", {})

        extracted_data = {"Stock Ticker": symbol}

        if module == "1":
            extracted_data["52 Week High"] = summary_detail.get("fiftyTwoWeekHigh", {}).get("raw", "N/A")
        elif module == "2":
            extracted_data["52 Week Low"] = summary_detail.get("fiftyTwoWeekLow", {}).get("raw", "N/A")
        elif module == "3":
            extracted_data["Return on Assets (ROA)"] = financial_data.get("returnOnAssets", {}).get("raw", "N/A")
        else:
            print("Invalid choice.")
            return

        df = pd.DataFrame([extracted_data])
        print(df.to_string(index=False))

    else:
        print("Stock data error")


def task2_2(apikey):
    url = "https://yfapi.net/v1/finance/trending/US"
    headers = {'x-api-key': apikey}

    response = requests.get(url, headers=headers)
    data = response.json()

    if "finance" in data and "result" in data["finance"] and len(data["finance"]["result"]) > 0:
        trending_symbols = [stock["symbol"] for stock in data["finance"]["result"][0]["quotes"]]

        query_url = "https://yfapi.net/v6/finance/quote"
        querystring = {"symbols": ",".join(trending_symbols)}
        response = requests.get(query_url, headers=headers, params=querystring)
        stock_data = response.json()

        if "quoteResponse" in stock_data and "result" in stock_data["quoteResponse"]:
            trending_stocks = []

            for stock in stock_data["quoteResponse"]["result"]:
                symbol = stock.get("symbol", "N/A")
                company = stock.get("longName", "N/A")
                price = stock.get("regularMarketPrice", "N/A")
                high_52 = stock.get("fiftyTwoWeekHigh", "N/A")
                low_52 = stock.get("fiftyTwoWeekLow", "N/A")

                price = f"${price:.2f}" if isinstance(price, (int, float)) else "N/A"
                high_52 = f"${high_52:.2f}" if isinstance(high_52, (int, float)) and high_52 < 10000 else "N/A"
                low_52 = f"${low_52:.2f}" if isinstance(low_52, (int, float)) else "N/A"

                trending_stocks.append({
                  "Stock Ticker": symbol,
                  "Company": company,
                  "Current Market Price": price,
                  "52 Week High": high_52,
                  "52 Week Low": low_52
              })

            df = pd.DataFrame(trending_stocks)

            print("\n3. Trending Stocks DataFrame:\n")
            print(df.to_string(index=False))

        else:
            print("Trending stocks error")
    else:
        print("Trending stocks error")

# Task 1
stock_symbols = input("1. Enter 2+ stock symbols (separate using commas): \n").strip().upper().split(",")
task1(stock_symbols, apikey)

# Task 2.1
symbol = input("\n2. Enter a stock symbol:").strip().upper()
print("Choose the data you want to fetch:")
print("1) 52 Week High")
print("2) 52 Week Low")
print("3) Return on Assets (ROA)")
module = input("Enter number of the module you are looking for: \n")
task2_1(symbol, module, apikey)

# Task 2.2
task2_2(apikey)

1. Enter 2+ stock symbols (separate using commas): 
a
Stock Ticker: A, Company: Agilent Technologies, Inc., Current Market Price: $134.47

2. Enter a stock symbol:a
Choose the data you want to fetch:
1) 52 Week High
2) 52 Week Low
3) Return on Assets (ROA)
Enter number of the module you are looking for: 
2
Stock Ticker  52 Week Low
           A       124.16

3. Trending Stocks DataFrame:

Stock Ticker                              Company Current Market Price 52 Week High 52 Week Low
        NVDA                   NVIDIA Corporation              $131.28      $153.13      $75.61
        SNOW                       Snowflake Inc.              $166.19      $233.88     $107.13
         CRM                     Salesforce, Inc.              $307.33      $369.00     $212.00
        IONQ                           IonQ, Inc.               $29.93       $54.74       $6.22
        BYND                    Beyond Meat, Inc.                $3.56       $12.12       $3.30
        MARA                  MA